# 📚 Etude pour [PULUMOX](https://github.com/nnosal/pulumox-template) : Ajouter un moteur de template yaml imbriqué afin de pouvoir injecter des variables dynamiques
> @date: 14/08/2025

In [ ]:
!pip install pyyaml
import os, yaml

## 1. ✅ Utiliser [adobe/himl](https://github.com/adobe/himl)
> Syntaxe: `k8s` ; ✅(merge) / ⚠️(dynamique-via-env-vars)
---
La solution la plus avancée pour proposer un pattern de fusion par dossier, la substitution ne peux être fait que via variable d'environnement (ce qui dans notre cas de call async peux être problématique donc sera écarté) sinon les méthodes disponibles sont uniquement ceux build-in (secret manager, ...)

In [ ]:
# 1. Création de la structure YAML
def create_demo_yaml_structure(base='./_test/himl'):
    os.makedirs(base, exist_ok=True)
    files = {
        'base.yaml': """
---
name: TOTO
version: 1
kubeconfig_location: "{{env(KUBECONFIG)}}"
""",
        'win11.yaml': """
---
name: TITI
os: linux
""",
    }
    for fname, content in files.items():
        with open(f'{base}/{fname}', 'w') as f:
            f.write(content)
    print("Structure et contenu YAML créés avec himl.")

create_demo_yaml_structure()

Structure et contenu YAML créés avec himl.


### 1/ Via env

In [ ]:
!pip install himl Jinja2
from himl import ConfigProcessor

os.environ["KUBECONFIG"] = "/home/user/.kube/config"

config_processor = ConfigProcessor()
merged = config_processor.process(
    path="./_test/himl",
    output_format="dict",
    print_data=False
)
print(merged)
# => /home/user/.kube/config


OrderedDict([('name', 'TITI'), ('version', 1), ('kubeconfig_location', '/home/user/.kube/config'), ('os', 'linux')])


### 2/ Via load+jinja2 (ébauche wtf)

In [ ]:
from jinja2 import Environment, StrictUndefined

os.environ["windows_url"] = "http://www.facebook.com"

# 2. Variables à injecter
variables = {
    "windows_url": "https://downloads.microsoft.com/ma-nouvelle-iso-windows.iso"
}

# 3. Prétraitement Jinja2 avant fusion
try:
  env = Environment(undefined=StrictUndefined)
  env.globals.update(variables)

  for fname in os.listdir('./_test/himl'):
      path = os.path.join('./_test/himl', fname)
      if os.path.isfile(path) and fname.endswith('.yaml'):
          with open(path) as f:
              rendered = env.from_string(f.read()).render()
          with open(path, 'w') as f:
              f.write(rendered)

  # 4. Fusion avec himl
  config_processor = ConfigProcessor()
  merged_config = config_processor.process(
      path='./_test/himl',
      output_format="dict",  # retourne un dict Python
      print_data=False
  )

  print(merged_config)

except Exception as e:
  print(e)
  print("Erreur causée par 'kubeconfig_location: \"{{env(KUBECONFIG)}}\"', la structure env d'HIML est incompatible avec celle de jinja2.")

'env' is undefined
Erreur causée par 'kubeconfig_location: "{{env(KUBECONFIG)}}"', la structure env d'HIML est incompatible avec celle de jinja2.


## 2. ✅ Utiliser [zerwes/hiyapyco](https://github.com/zerwes/hiyapyco)
> Syntaxe: `jinja2`.
---
Permet de merge (même si n'a pas de pattern par dossier il faut la coder) + injection de variable dynamique grâce à Jinja2.

In [ ]:
!pip install hiyapyco Jinja2
def create_demo_yaml_structure(base='./_test/hiyapyco'):
    os.makedirs(f'{base}/include.d', exist_ok=True)
    files = {
        'base.yaml': """
---
name: TOTO
version: 1
iso:
  url: "{{ windows_url }}"
""",
        'win11.yaml': """
---
name: TITI
os: linux
""",
    }
    for f, c in files.items():
        with open(f'{base}/{f}', 'w') as file: file.write(c)
    print("Structure et contenu YAML créés.")

create_demo_yaml_structure()

#!/usr/bin/env uv run --script
# /// script
# dependencies = [
#   "hiyapyco", "Jinja2"
# ]
# ///
import hiyapyco
from jinja2 import Environment, Undefined, DebugUndefined, StrictUndefined

# 1. Définir la nouvelle valeur à insérer
windows_url = "https://downloads.microsoft.com/ma-nouvelle-iso-windows.iso"

# 2. Définir les variables Jinja2 dans un dictionnaire
variables = {
    'windows_url': windows_url
}

# 3. Charger le fichier YAML avec Hiyapyco et fournir le dictionnaire de variables
# Hiyapyco va automatiquement rendre le template en utilisant les variables fournies
hiyapyco.jinja2env = Environment(undefined=StrictUndefined) # ou =Undefined pour que pas d'erreur + value="" # =StrictUndefined == srict
hiyapyco.jinja2env.globals.update(variables)

hiyapyco_method=hiyapyco.METHOD_MERGE
#hiyapyco_method=hiyapyco.METHOD_SIMPLE
#hiyapyco_method=hiyapyco.METHOD_SUBSTITUTE
config = hiyapyco.load('_test/hiyapyco/base.yaml', '_test/hiyapyco/win11.yaml',
    interpolate=True,
    method=hiyapyco_method,
    #none_behavior=hiyapyco.NONE_BEHAVIOR_OVERRIDE,
    )

# La variable a été remplacée au moment du chargement
print(config)
# Output:
# {'iso': {'url': 'https://downloads.microsoft.com/ma-nouvelle-iso-windows.iso'}}

Structure et contenu YAML créés.
OrderedDict([('name', 'TITI'), ('version', 1), ('iso', OrderedDict([('url', 'https://downloads.microsoft.com/ma-nouvelle-iso-windows.iso')])), ('os', 'linux')])


## 3. ✅ Utiliser [x41lakazam/mergeconfigs](https://github.com/x41lakazam/mergeconfigs)
> Syntaxe: `#{extends,load,include}`, `${filename:var}`.

In [ ]:
from mergeconfigs.config_builder import build_config

def create_demo_yaml_structure(base='./_test/mergeconfigs'):
    os.makedirs(base, exist_ok=True)
    files = {
        '_base.yaml': """#load variables.yaml
---
name: BASE
version: 1
iso:
  url: "${variables@windows_url}"
""",
        'win11.yaml': """
#extends _base.yaml
---
name: WIN11
os: windows
""",
        'variables.yaml': """
---
windows_url: "https://downloads.microsoft.com/ma-nouvelle-iso-windows.iso"
""",
        'win10.yaml': """
#extends _base.yaml
---
name: WIN10
os: windows
""",
        'variables.yaml': """
---
windows_url: "https://downloads.microsoft.com/ma-nouvelle-iso-windows.iso"
"""
    }
    for f, c in files.items():
        with open(f'{base}/{f}', 'w') as file:
            file.write(c)
    print("Structure et contenu YAML créés.")

create_demo_yaml_structure()

# Utilisation directe de l’API interne
merged_dict = build_config(
    filename="win11.yaml",           # fichier de départ
    workdir="./_test/mergeconfigs",  # répertoire de travail
    env="."                          # environnement
)

# Affichage du résultat fusionné
print(yaml.dump(merged_dict, sort_keys=False, allow_unicode=True))


Structure et contenu YAML créés.
name: WIN11
version: 1
iso:
  url: https://downloads.microsoft.com/ma-nouvelle-iso-windows.iso
os: windows



## 4. ❌/🔮 Utiliser [valentingol/cliconfig](https://github.com/valentingol/cliconfig/)
> Syntaxe: `@{merge_,copy,def,type,select,delete,new,dict}`.
---
N'autorise des args que sous-forme de cli... la lib pourrait être intéréssante mais il va falloir l'adapter + pull_request

In [ ]:
!pip install cliconfig
def create_demo_yaml_structure(base='./_test/cliconfig'):
    os.makedirs(f'{base}/include.d', exist_ok=True)
    files = {
        'base.yaml': """
---
name: TOTO
version: 1
iso:
  url: "{{ windows_url }}"
""",
        'win11.yaml': """
---
name: TITI
os: linux
""",
    }
    for f, c in files.items():
        with open(f'{base}/{f}', 'w') as file: file.write(c)
    print("Structure et contenu YAML créés.")

create_demo_yaml_structure()

#!/usr/bin/env uv run --script
# /// script
# dependencies = [
#   "cliconfig"
# ]
# ///
from cliconfig import make_config
jinja_vars = {
    "windows_url": "https://example.com/windows.iso"
}
config = make_config( '_test/cliconfig/base.yaml', '_test/cliconfig/win11.yaml', no_cli=True )
# can't add vars...
print(config)

Structure et contenu YAML créés.
INFO - [CONFIG] Merged 2 default config(s), 0 additional config(s) and 0 CLI parameter(s).


INFO:cliconfig._logger:[CONFIG] Merged 2 default config(s), 0 additional config(s) and 0 CLI parameter(s).


Config({'name': 'TITI', 'version': 1, 'iso': {'url': '{{ windows_url }}'}, 'os': 'linux'}, ['ProcessCheckTags', 'ProcessMerge', 'ProcessCopy', 'ProcessDef', 'ProcessTyping', 'ProcessSelect', 'ProcessDelete', 'ProcessDict', 'ProcessNew'])


## 5. ❌ Utiliser [tanbro/pyyaml-include](https://github.com/tanbro/pyyaml-include)
> Syntaxe: `Linux '.d' suffix` ;  ✅(!include) / ❌(dynamic-vars)
---
En apparence `.d` mais c'est l'exemple qui le présente tel quel... il s'agit d'un simple fichier `master` qui nécessite d'inclure les règles d'inclusions, `yaml_include` est juste une classe d'include pour `add_constructor`, son avantage serait pour sa déclinaisons fsspec (http,s3,etc) mais inutile dans mon cas.

In [ ]:
!pip install pyyaml-include
import yaml
import yaml_include
import os

def create_demo_yaml_structure(base='./_test/pyyaml-include'):
    os.makedirs(f'{base}/include.d', exist_ok=True)
    files = {
        'include.d/1.yaml': """
---
name: "1"
""",
        'include.d/2.yaml': """
---
name: "2"
""",
        '0.yaml': """
---
ab: !inc [./include.d/*.yml]
site: !url
"""
    }
    for f, c in files.items():
        with open(f'{base}/{f}', 'w') as file: file.write(c)
    print("Structure et contenu YAML créés.")

create_demo_yaml_structure()

# ######

#!/usr/bin/env uv run --script
# /// script
# dependencies = [
#   "pyyaml-include"
# ]
# ///
import yaml
import yaml_include

# add the !inc tag to SafeLoader
yaml.SafeLoader.add_constructor("!inc", yaml_include.Constructor(base_dir='_test/pyyaml-include'))

# Define a custom loader function for !url
def my_url_loader(loader, node):
    return "http://www.google.fr"

# Add the !url constructor to SafeLoader
yaml.SafeLoader.add_constructor("!url", my_url_loader)

with open('_test/pyyaml-include/0.yaml') as f:
  data = yaml.safe_load(f)

print(data)

Structure et contenu YAML créés.
{'ab': [], 'site': 'http://www.google.fr'}


## 6. ❌ Utiliser [lbovet/yglu](https://github.com/lbovet/yglu)
> Syntaxe: `YAQL`
---
N'autorise son execution que sous-forme de cli ou en tout cas son usage n'est pas documenté.

## 7. ❌ Utiliser [yte](https://github.com/yte-template-engine/yte) -
> Syntaxe: Python.
> ❌(merge) / ✅(dynamic-vars)
---
Il n'a pas de capacité de merging, il s'agit que d'un moteur de template pour YAML, même si très sympa, ça reste un Jinja2-like.

In [ ]:
#!/usr/bin/env uv run --script
# /// script
# dependencies = [
#   "yte"
# ]
# ///
!pip install yte
from yte import process_yaml

# Variables à injecter dans le template
variables = {
    "url_virtio_iso": "https://exemple.com/virtio-win.iso"
}

# Charger et rendre le fichier de base
win_base_yml = """
---
vmimgs:
  - name: test-latest-virtio-win
    file_name: test-virtio-win.iso
    url: "? url_virtio_iso"
    node_name: "auto"
    datastore_id: "local"

vmtpls2:
  - name: "tpl--windows"
    bios: "ovmf"
    machine: "pc-q35-6.2"
    keyboard_layout: "fr"
    description: "Windows-virtio master-VM to clone"
    tags: ["tpl"]
    node_name: "mpro"
    resource_name: "tpl--windows"
    template: true
    vm_id: 9000
    vga:
      type: "std"
      memory: 128
    boot_orders:
      - "ide0"
      - "sata1"
      - "ide1"
      - "ide3"
    cdrom:
      interface: "ide1"
      file_id: "local:iso/2_latest-virtio-win.iso"
    on_boot: false
"""
base_yaml = process_yaml(win_base_yml, variables=variables)
win11_yml = """
---
vmtpls2:
  - name: "tpl--windows11"
"""
override_yaml = process_yaml(win11_yml, variables=variables)

# Fusion : remplacer/ajouter les clés de override dans base
merged = {**base_yaml, **override_yaml}

# Afficher le YAML final
print(yaml.safe_dump(merged, sort_keys=False, allow_unicode=True))


vmimgs:
- name: test-latest-virtio-win
  file_name: test-virtio-win.iso
  url: https://exemple.com/virtio-win.iso
  node_name: auto
  datastore_id: local
vmtpls2:
- name: tpl--windows11



## 8. ✅🤓 Utiliser simplement yaml.add_constructor?
---
Finalement revient à construire son propre constructeur (native à la lib yaml)... je crois que ça va être le plus simple même si personne n'a fait quelque chose de mieux.

In [ ]:
import yaml

# Exemple de fonction multiple args
def windlurl(name, lang, arch):
    return f"https://example.com/download?name={name}&lang={lang}&arch={arch}"

# Constructeur YAML pour !windlurl
yaml.SafeLoader.add_constructor('!windlurl', lambda loader, node: windlurl(*loader.construct_sequence(node)))

# YAML
yaml_content = """
file: !windlurl ["❤️ Windows 11", "French", "arm64"]
"""

data = yaml.safe_load(yaml_content)
print(data['file'])

https://example.com/download?name=❤️ Windows 11&lang=French&arch=arm64


# Structure souhaitée
---

## Windows
- templates/windows/_base.yaml
- templates/windows/10.yaml
- templates/windows/11.yaml

## Mac (->2020 Era)
- templates/mac/_base.yaml
- templates/mac/11-big-sur.yaml
- templates/mac/12-monterey.yaml
- templates/mac/13-ventura.yaml
- templates/mac/14-sonoma.yaml
- templates/mac/15-sequoia.yaml
- templates/mac/26-tahoe.yaml

## Mac (10. -> 2010 Era - x64_app_store )
- templates/mac/_base10.yaml
- templates/mac/10.7-lion.yaml
- templates/mac/10.8-mountain-lion.yaml
- templates/mac/10.9-mavericks.yaml
- templates/mac/10.10-yosemite.yaml
- templates/mac/10.11-el-capitan.yaml
- templates/mac/10.12-sierra.yaml
- templates/mac/10.13-high-sierra.yaml
- templates/mac/10.14-mojave.yaml
- templates/mac/10.15-catalina.yaml

## Linux
- templates/windows/_base.yaml
- templates/windows/ubuntu.yaml
- templates/windows/debian.yaml
- templates/windows/centos-stream.yaml
- templates/windows/linux-mint.yaml
- templates/windows/manjaro.yaml


# Conclusion
---
1. `Adobe/HIML` est plus dans l'esprit de ce que je veux faire, une configuration par structure, cependant son injection dynamique est trop limitative (via variable d'env.).
2. `Cliconfig`, permet de faire des fusions très précises de façon purement déclarative (voir trop) mais n'a aucun mécanisme d'injection de variables en dehors de sa version cli.
3. 👍 **`hiyapyco`, reste le plus adapté, gestion native de fusion + templating Jinja2**. Par exemple `yte`, propore un moteur de templating Yaml intéréssant mais il reste nécessaire de faire une fusion non-deep comme `merged = {**base_yaml, **override_yaml}`.
4. 👍⚠️ `Mergeconfig` si détourné de sa version cli, permet de gérer une structure + interpolation, sauf que la lib n'est plus maintenu.

`3. Mergeconfig` ressemble le plus à ce que je veux en terme de structure + fusion (`#extends _base.yaml` ), je vais partir sur cette base ou soit sur `2. hiyapyco` (`_base.yml.j2`) ou soit alors coder simplement mon constructeur yaml (`8. yaml.add_constructor` + `merged = {**base_yaml, **override_yaml}`)
